<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/Test_pipeline_model_tmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [79]:
# 경로 설정 
import os
os.chdir('/content/drive/MyDrive/A2W/데이터/data_submission')

In [3]:
import pandas as pd
import csv

In [80]:
# Merged_data_file
Merged_2020 = pd.read_csv('merge_s_h_a20.csv')
Merged_2020.head()

,tm,stnId,stnNm,umd,sgg,umd2,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT
0,2020-01-01,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,2.1,-0.6,3.8,0.0,0.0,0.00,0.0,7.9,4.0,1.8,46.0,56.4,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
1,2020-01-02,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,4.7,2.7,7.2,0.0,0.0,0.00,0.0,9.9,6.4,2.0,40.0,50.8,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
2,2020-01-03,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,5.9,4.1,8.5,0.0,0.0,0.00,0.0,11.7,7.1,2.3,38.0,51.6,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
3,2020-01-04,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,5.0,2.7,8.1,0.0,0.0,2.23,1.5,18.1,9.1,4.0,40.0,55.6,1.5,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
4,2020-01-05,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,4.4,2.7,6.2,0.0,0.0,5.75,4.7,8.2,5.7,2.2,55.0,73.5,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0


In [5]:
# Test 검증 데이터셋
test_input = pd.read_csv('input.csv', encoding = 'cp949')
test_input.head()

,date,sd,sgg,umd,1day,1day_yn,2day,2day_yn
0,2020-06-10,경상남도,사천시,곤명면,2020-06-11,NaN,2020-06-12,NaN
1,2020-06-11,경상남도,사천시,곤명면,2020-06-12,NaN,2020-06-13,NaN
2,2020-06-12,경상남도,사천시,곤명면,2020-06-13,NaN,2020-06-14,NaN
3,2020-06-10,경상남도,사천시,곤양면,2020-06-11,NaN,2020-06-12,NaN
4,2020-06-11,경상남도,사천시,곤양면,2020-06-12,NaN,2020-06-13,NaN


In [6]:
# searching data 찾아야 됨
def search_data(sgg, umd, date):
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    specific = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    return specific

'''
input 예시
input_date = '2020-06-11' # 날짜 단위
input_sgg = '울릉군' # 시 단위
input_umd = '울릉읍' # 읍면동 단위
'''

"\ninput 예시\ninput_date = '2020-06-11' # 날짜 단위\ninput_sgg = '울릉군' # 시 단위\ninput_umd = '울릉읍' # 읍면동 단위\n"

In [9]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.externals import joblib

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


from xgboost import plot_importance
from xgboost import XGBClassifier

In [10]:
os.chdir('/content/drive/MyDrive/A2W/데이터/model')

In [11]:
xgb1 = joblib.load('xgb_best_model_1day_yn.pkl') 
xgb2 = joblib.load('xgb_best_model_2day_yn.pkl')

In [26]:
xgb1_cols = xgb1.get_booster().feature_names
xgb2_cols = xgb2.get_booster().feature_names

In [97]:
# 훈련된 모델로 결과 도출 : day1 함수
def day1_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day1 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn','hr1MaxRn','sumRnDur',
                                    'sumRn','maxInsWs','maxWs','avgWs','minRhm','avgRhm','n99Rn','PRRCK_LARG',
                                    'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C',
                                    'SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD',
                                    'KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    # fillna_test_specific_day1 = test_specific_day1.fillna(0)
    # df = pd.DataFrame[fillna_test_specific_day1]
    model_pred = xgb1.predict(test_specific_day1)
    return model_pred

In [93]:
test_day1 = Merged_2020.loc[100]
test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn','hr1MaxRn','sumRnDur',
                                    'sumRn','maxInsWs','maxWs','avgWs','minRhm','avgRhm','n99Rn','PRRCK_LARG',
                                    'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C',
                                    'SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD',
                                    'KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]

In [120]:
length_test_input = len(test_input.index)

# for index in range(1):
input_date = test_input['date'][1]
input_sgg = test_input['sgg'][1]
input_umd = test_input['umd'][1]

text = input_sgg + ' ' + input_umd

test_day1 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(input_date)]
test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn','hr1MaxRn','sumRnDur',
                                    'sumRn','maxInsWs','maxWs','avgWs','minRhm','avgRhm','n99Rn','PRRCK_LARG',
                                    'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C',
                                    'SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD',
                                    'KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]

# test_specific_day1 = test_specific_day1[xgb1.get_booster().feature_names]
model_pred = xgb1.predict(test_specific_day1.values)

test_input['1day_yn'][1] = model_pred

ValueError: ignored

In [82]:
length_test_input = len(test_input.index)

# day_1 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['1day_yn'][index] = day1_data_test(input_sgg, input_umd, input_date)
        
test_input.head()

ValueError: ignored

In [45]:
# 훈련된 모델로 결과 도출 : day2 함수
def day2_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day2 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day2 = test_day2[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 
                                    'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 
                                    'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C',
                                    'SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD',
                                    'KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day2 = test_specific_day2.fillna(0)
    model_pred = xgb2.predict(fillna_test_specific_day2.values)
    return model_pred

In [ ]:
# day_2 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['2day_yn'][index] = day2_data_test(input_sgg, input_umd, input_date)
        
# print(search_data(input_sgg, input_umd, input_date))

In [ ]:
test_input["1day_yn"].value_counts()

0.0    3380
1.0      30
Name: 1day_yn, dtype: int64

In [ ]:
test_input["2day_yn"].value_counts()

0.0    3410
Name: 2day_yn, dtype: int64

In [ ]:
test_input.to_csv("xgb1_best_model.csv", index=False, encoding='cp949')